In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!pip install deeppavlov
!pip install transformers
!pip install finbert-embedding
!pip install spacy
!pip install xgboost
!pip install fuzzywuzzy
!pip install spacy[transformers]
!python -m spacy download en_core_web_trf


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.2 MB/s 
     |████████████████████████████████| 145 kB 52.2 MB/s 
     |████████████████████████████████| 42 kB 359 kB/s 
     |████████████████████████████████| 59 kB 3.0 MB/s 
     |████████████████████████████████| 55 kB 1.6 MB/s 
     |████████████████████████████████| 54 kB 905 kB/s 
     |████████████████████████████████| 57 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 12.4 MB/s 
     |████████████████████████████████| 63 kB 895 kB/s 
     |████████████████████████████████| 80 kB 3.7 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=55ae838ee00134557bda3544c4e37334b610342cf85413e68ee8d848238c7847
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built 

In [3]:
%cd '/content/gdrive/MyDrive/Capstone/t2s'
from utilities import *

/content/gdrive/MyDrive/Capstone/t2s


2022-12-03 15:35:12.424 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.pickle to /root/.deeppavlov/downloads/query_prediction/query_prediction_eng.pickle
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.pickle to /root/.deeppavlov/downloads/query_prediction/query_prediction_eng.pickle
100%|██████████| 2.19M/2.19M [00:00<00:00, 3.15MB/s]
2022-12-03 15:35:14.511 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.tar.gz to /root/.deeppavlov/models/classifiers/query_prediction_eng.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/kbqa/wikidata/query_prediction_eng.tar.gz to /root/.deeppavlov/models/classifiers/query_prediction_eng.tar.gz
100%|██████████| 238M/238M [00:09<00:00, 25.9MB/s]
2022-12-03 15:35:24.395 INFO in 'dee

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at haisongzhang/roberta-tiny-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
def text2sparql(query,ontology):

  try:
    sparql_query_list=[]

    template_id,n_entities, query_templates=template_classification(query)#
    print(query_templates)

    if template_id==4 or template_id==2:
      #print(get_single_relation(query,ontology))
      relation,relation_class=get_single_relation(query,ontology)#
      print(relation)

      entity,entity_class=get_single_entity(query,ontology)#
      query_validity=check_query_validity(entity, relation, ontology)


      if (entity!="Not found" and relation!="Not found" ):
        if query_validity==True:  
          for query_template in query_templates:
            sparql_query_list.append(query_template.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")).replace("RELATION_CLASS",relation_class.replace(" ","_")).replace("RELATION",relation.replace(" ","_")))
        else: 
          q="SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> ?p ?o.}"
          sparql_query_list.append(q.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))

      else:
        sparql_query_list.append("Not found")

    if template_id==1:
      print("hello")
      entities, entities_class=get_entities(query,ontology)

      if len(entities)!=0:
        for i,entity in enumerate(entities):
          entity_class=entities_class[i]
          #print(entity, entity_class)
          sparql_query_list.append(query_templates.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))
      else:
        sparql_query_list.append("Not found")

    if template_id==5:
          relation,relation_class=get_single_relation(query,ontology)#
          entities, entities_class=get_entities(query,ontology)
          aggregation=get_aggregation_method(query)
          #print(aggregation)
          if (relation!="Not found" and relation!="Not found"):
            for i,entity in enumerate(entities):
              query_validity=check_query_validity(entity, relation, ontology)
              entity_class=entities_class[i]

              if query_validity==True:              
                for agg_method in aggregation:
                  #print(agg_method)
                  sparql_query_list.append(query_templates[0].replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")).replace("RELATION_CLASS",relation_class.replace(" ","_")).replace("RELATION",relation.replace(" ","_")).replace("AGGREGATION_METHOD",agg_method.replace(" ","_")))
              else:
                  q= "SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> ?p ?o.}"
                  sparql_query_list.append(q.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))
      
          else:
            sparql_query_list.append("Not found")

  except:
        entities, entities_class=get_entities(query,ontology)
        q= "SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> ?p ?o.}"

        if len(entities)!=0:
            for i,entity in enumerate(entities):
              entity_class=entities_class[i]
              #print(entity, entity_class)
              sparql_query_list.append(q.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))
        else:
            sparql_query_list.append("Not found")

  
  
  return sparql_query_list

In [18]:
query1="Which crypto costs the most?" #gives better result on stanford
query2="Ramit Sawhney"
query3="What does Ramit Sawhney do?"
query4="Who is Jonathan Donier?"
query5="What paper was written by Ramit Sawhney?"
query6="How many papers has Ramit Sawhney written?"
query7="List the papers written by Ramit Sawhney"
query8="How to regularize GML models?"
query9="How many crypto miners are openly available?" # won't work
query10="Macroscale ODE model was derived by whom?"
query11="Who derived macroscaled ODE model?" # wrong classification
query12="What do venture capitalists do?"
text2sparql(query12, "allen")

['SELECT DISTINCT ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> <http://crypto.org/RELATION_CLASS/RELATION> ?o.}', 'SELECT DISTINCT ?s WHERE { ?s <http://crypto.org/RELATION_CLASS/RELATION> <http://crypto.org/ENTITY_CLASS/ENTITY>.}']
do


/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


['SELECT DISTINCT ?p ?o WHERE { <http://crypto.org/CONTRACT/venture_capital> ?p ?o.}']